In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('medium_posts.csv')

In [6]:
df['published'] = pd.to_datetime(df['published'])

In [7]:
df.sort_values(by=['published']).head(n=3)

published      domain  \
40243 1970-01-01 00:00:00.001  medium.com   
50931 1970-01-01 00:00:00.001  medium.com   
37395 1970-01-18 05:11:46.500  medium.com   

                                                     url  
40243  https://medium.com/@ikaella/melon-rebranding-b...  
50931                            https://medium.com/iiot  
37395  http://www.novosti.rs/%D0%B2%D0%B5%D1%81%D1%82...

In [8]:
df = df[['published', 'url']].dropna().drop_duplicates()

In [9]:
df = df[(df['published'] > '2012-08-15') & (df['published'] < '2017-06-26')].sort_values(by=['published'])
df.head(n=3)

published  \
24630 2012-08-15 00:25:03.373   
24631 2012-08-15 00:25:29.419   
17811 2012-08-15 00:34:59.502   

                                                     url  
24630  https://medium.com/launch-day/jean-attempts-to...  
24631  https://medium.com/launch-day/dan-and-kristin-...  
17811  https://medium.com/i-m-h-o/the-world-is-social...

In [10]:
df.tail(n=3)

published  \
62122 2017-06-25 23:36:01.171   
72471 2017-06-25 23:41:48.295   
83283 2017-06-25 23:51:43.000   

                                                     url  
62122  https://medium.com/push-the-pace/the-official-...  
72471  https://medium.com/parti-xyz-developers/%EA%B4...  
83283   http://www.johanr.com/blog/people-support-dreams

In [11]:
aggr_df = df.groupby('published')[['url']].count()
aggr_df.columns = ['posts']

In [12]:
aggr_df.head(n=3)

posts
published                     
2012-08-15 00:25:03.373      1
2012-08-15 00:25:29.419      1
2012-08-15 00:34:59.502      1

In [13]:
daily_df = aggr_df.resample('D').apply(sum)
daily_df.head(n=3)

posts
published        
2012-08-15     16
2012-08-16     11
2012-08-17      4

In [14]:
# EDA Visual

from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

# Initialize plotly
init_notebook_mode(connected=True)

In [15]:
def plotly_df(df, title=''):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode='lines')
    data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

In [16]:
plotly_df(daily_df, title='Posts on Medium (daily)')

In [17]:
weekly_df = daily_df.resample('W').apply(sum)

In [18]:
plotly_df(weekly_df, title='Posts on Medium (weekly)')

In [19]:
daily_df = daily_df.loc[daily_df.index >= '2015-01-01']
daily_df.head(n=3)

posts
published        
2015-01-01      8
2015-01-02     11
2015-01-03     11

In [20]:
from fbprophet import Prophet

import logging
logging.getLogger().setLevel(logging.ERROR)

In [21]:
df = daily_df.reset_index()
df.columns = ['ds', 'y']
df.tail(n=3)

ds    y
904 2017-06-23  421
905 2017-06-24  277
906 2017-06-25  253

In [22]:
prediction_size = 30
train_df = df[:-prediction_size]
train_df.tail(n=3)

ds    y
874 2017-05-24  375
875 2017-05-25  298
876 2017-05-26  269

In [23]:
m = Prophet()
m.fit(train_df);

In [24]:
future = m.make_future_dataframe(periods=prediction_size)
future.tail(n=3)

ds
904 2017-06-23
905 2017-06-24
906 2017-06-25

In [25]:
forecast = m.predict(future)
forecast.tail(n=3)

ds       trend  trend_lower  trend_upper  yhat_lower  yhat_upper  \
904 2017-06-23  277.992028   277.512648   278.419131  255.295825  305.138754   
905 2017-06-24  278.710324   278.204716   279.170271  218.795589  267.324689   
906 2017-06-25  279.428621   278.888042   279.916124  225.409202  271.847229   

     additive_terms  additive_terms_lower  additive_terms_upper  \
904        2.242612              2.242612              2.242612   
905      -35.065401            -35.065401            -35.065401   
906      -31.432830            -31.432830            -31.432830   

     multiplicative_terms  multiplicative_terms_lower  \
904                   0.0                         0.0   
905                   0.0                         0.0   
906                   0.0                         0.0   

     multiplicative_terms_upper     weekly  weekly_lower  weekly_upper  \
904                         0.0   3.473493      3.473493      3.473493   
905                         0.0 -34.023843    -34.023843    -34.023843   
906                         0.0 -30.580216    -30.580216    -30.580216   

       yearly  yearly_lower  yearly_upper        yhat  
904 -1.230880     -1.230880     -1.230880  280.234641  
905 -1.041558     -1.041558     -1.041558  243.644924  
906 -0.852614     -0.852614     -0.852614  247.995791

In [26]:
m.plot(forecast);

In [27]:
m.plot_components(forecast);

In [28]:
# Forecast quality evaluation

print(', '.join(forecast.columns))

ds, trend, trend_lower, trend_upper, yhat_lower, yhat_upper, additive_terms, additive_terms_lower, additive_terms_upper, multiplicative_terms, multiplicative_terms_lower, multiplicative_terms_upper, weekly, weekly_lower, weekly_upper, yearly, yearly_lower, yearly_upper, yhat


In [29]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [30]:
cmp_df = make_comparison_dataframe(df, forecast)
cmp_df.tail(n=3)

yhat  yhat_lower  yhat_upper    y
ds                                                 
2017-06-23  280.234641  255.295825  305.138754  421
2017-06-24  243.644924  218.795589  267.324689  277
2017-06-25  247.995791  225.409202  271.847229  253

In [31]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    
    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Recall that we held out the values of the last `prediction_size` days
    # in order to predict them and measure the quality of the model. 
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [32]:
for err_name, err_value in calculate_forecast_errors(cmp_df, prediction_size).items():
    print(err_name, err_value)

MAPE 22.566119266921035
MAE 69.62700966533414


In [34]:
# Visualizing

def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Lower Bound', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="gray"))
    upper_bound = create_go('Upper Bound', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="gray"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Forecast', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Actual', 'y', num_values,
                       marker=dict(color="red"))
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='Posts'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

show_forecast(cmp_df, prediction_size, 100, 'New posts on Medium')

In [35]:
# Box Cox Transfermation

def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [36]:
train_df2 = train_df.copy().set_index('ds')

In [37]:
train_df2['y'], lambda_prophet = stats.boxcox(train_df2['y'])
train_df2.reset_index(inplace=True)

In [38]:
m2 = Prophet()
m2.fit(train_df2)
future2 = m2.make_future_dataframe(periods=prediction_size)
forecast2 = m2.predict(future2)

In [39]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column], lambda_prophet)

In [40]:
cmp_df2 = make_comparison_dataframe(df, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, prediction_size).items():
    print(err_name, err_value)

MAPE 11.765883751504633
MAE 39.819039511397186


In [41]:
show_forecast(cmp_df, prediction_size, 100, 'No transformations')
show_forecast(cmp_df2, prediction_size, 100, 'Box–Cox transformation')